Problem Solving Process
================

#### 1. Load Data
#### 2. Checking Datas
#### 3. Explanatory Data Analysis
#### 4. Feature Engineering
#### 5. Model Selection
#### 6. Evaluation and Application

## 1. Load Data

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## 2. Checking Datas

### 2.1 Basic Data Checking  
***train, test 데이터 모두 확인해보기***

In [ ]:
train.head()
train.tail()
train.shape
train.info()
train.describe()

test.head()
test.tail()
test.shape
test.info()
test.describe()

rows = train.shape[0]
columns = train.shape[1]
print('The train dataset contains {} rows and {} columns'.format(rows, columns))

### 2.2 Checking Missing Data  
***train, test 데이터 모두 확인해보기***  
***msno로 시각화 해보기*** -> (누락된 데이터가 너무 많으면 사용하지 않음!)

In [ ]:
for col in df_train.columns:
    msg = 'column : {:>10}\t Percent of Nan value : {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)
    
for col in df_test.columns:
    msg = 'column : {:>10}\t Percent of Nan value : {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

In [ ]:
msno.matrix(df=df_train.iloc[:, :], figsize=(8,8), color=(0.8, 0.5, 0.2))

msno.matrix(df=df_train.iloc[:, :], figsize=(8,8), color=(0.8, 0.5, 0.2))

msno.bar(df=df_test.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

### 2.3 Checking Target Label  
***0,1로 분류 가능한 레이블이 어떤 분포를 가지고 있는지 시각적으로 확인해보기***   
-> 극단적인 분포면 도움이 되지 않음..

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (18,8))

df_train['Survived'].value_counts().plot.pie(explode = [0, 0.1], 
                                             autopct='%1.1f%%',
                                            ax=ax[0],
                                            shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

### 2.4 Checking Data Types 

***categorical, ordinal, continuous***   
***MetaData 만들기***

In [ ]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

### 2.5 Checking for outliers

In [ ]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])

## 3. Explanatory Data Analysis

### 3.1 Checking number and ratio between all the features and target label

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).count()
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum()
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

### 3.2 Visualize the relation between all the features and target label

***bar, pie, distplot, countplot, violinplot, kdeplot 등을 활용***

In [ ]:
y_position = 1.02
fig, ax = plt.subplots(1,2,figsize=(18,8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass : Survived vs Dead', y=y_position)
plt.show()